In [7]:
import numpy as np
import dadi, random, pickle
import os, sys
sys.path.append(os.path.join(os.getcwd(), '..')) # this is the ml_dadi dir
import data_manip, ml_models
from data_manip import generating_data
from ml_models import rfr_train, mlpr_train

In [8]:
# generate a list of theta values to run scaling and add variance
theta_list = [1,10000,1000,100] # order of increase variance

In [9]:
# designate demographic model, sample size, and extrapolation grid 
func = dadi.Demographics1D.two_epoch
ns = [20]
pts_l = [40, 50, 60]

In [10]:
# designate which param to be in log scale
logs = [True, False]

In [5]:
# # Generate parameter list for training: exclude params where T/nu > 5 version
# # using log scale for nu
# train_params = [(nu,T) for nu in np.linspace(-2, 2, 25)
#                       for T in np.linspace(0.1, 2, 24) if T/10**nu <= 5]
# # print training set info 
# print('n_samples training: ', len(train_params))
# print('Range of training params:', min(train_params), 'to', max(train_params))
# print('Theta list:', theta_list)
# # Make a list of training data dictionaries, one dictionary for each theta case
# list_train_dict=generating_data(train_params, theta_list, func, ns, pts_l, logs)
# pickle.dump(list_train_dict, open('data/new_func/train_data', 'wb'), 2)

n_samples training:  410
Range of training params: (-1.6666666666666667, 0.1) to (2.0, 2.0)
Theta list: [1, 10000, 1000, 100]


In [5]:
# Load new datasets for training
list_train_dict = pickle.load(open('data/new_func/train_data','rb'))

In [6]:
# Train RFR 
list_rfr = [rfr_train(train_dict) for train_dict in list_train_dict]
pickle.dump(list_rfr, open('data/new_func/list_rfr', 'wb'), 2)

In [7]:
# Train MLPR with adam solver (default)
list_mlpr_adam = [mlpr_train(train_dict, max_iter=500) 
                        for train_dict in list_train_dict]
pickle.dump(list_mlpr_adam, open('data/new_func/list_mlpr_adam', 'wb'), 2)

In [9]:
# Train MLPR with lbfgs solver
# Need large max_iter and take longer to run but perform the best for two_epoch
list_mlpr_lbfgs = [mlpr_train(train_dict, solver='lbfgs', max_iter=5000)
                        for train_dict in list_train_dict]
pickle.dump(list_mlpr_lbfgs, open('data/new_func/list_mlpr_lbfgs', 'wb'), 2)

In [10]:
# Generate Test Datasets
test_params = []
while len(test_params) < 100: 
# generate random nu and T within the same range as training data range
    nu = random.random() * 4 - 2 # nu in log scale
    T = random.random() * 1.9 + 0.1
    # exclude T/nu > 5
    if T/10**nu <= 5:
        params = (nu, T)
        test_params.append(params)
# print testing set info 
print('n_samples testing: ', len(test_params))
print('Range of testing params:', min(test_params), 'to', max(test_params))
print('Theta list:', theta_list)
# Make a list of test data dictionaries, one dictionary for each theta case
list_test_dict = generating_data(test_params, theta_list, func, ns, pts_l, logs)
# Save testing set as a pickle file
pickle.dump(list_test_dict, open('data/new_func/test_data', 'wb'), 2)

n_samples testing:  100
Range of testing params: (-1.2835012041008178, 0.15754354169567486) to (1.9973742028068813, 0.7008000830008293)
Theta list: [1, 10000, 1000, 100]


In [11]:
# Generate Test Datasets Full Range
test_params = []
while len(test_params) < 100: 
# generate random nu and T within the same range as training data range
    nu = random.random() * 4 - 2 # nu in log scale
    T = random.random() * 1.9 + 0.1
    # # exclude T/nu > 5
    # if T/10**nu <= 5:
    #     params = (nu, T)
    #     test_params.append(params)
    test_params.append((nu, T))

# print testing set info 
print('n_samples testing: ', len(test_params))
print('Range of testing params:', min(test_params), 'to', max(test_params))
print('Theta list:', theta_list)
# Make a list of test data dictionaries, one dictionary for each theta case
list_test_dict = generating_data(test_params, theta_list, func, ns, pts_l, logs)
# Save testing set as a pickle file
pickle.dump(list_test_dict, open('data/new_func/test_data_full', 'wb'), 2)

n_samples testing:  100
Range of testing params: (-1.9835824785940854, 1.169385319983439) to (1.9724244058327387, 1.7354305216161359)
Theta list: [1, 10000, 1000, 100]
